## 1. Instalando as bibliotecas necessárias para webscraping pelo Colab

* Driver estável do Chrome
* Selenium WebDriver

In [47]:
# Instala o Google Chrome oficial
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y ./google-chrome-stable_current_amd64.deb

# Instala o webdriver-manager para gerenciar o driver automaticamente
!pip install selenium webdriver-manager

--2026-02-17 16:57:16--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.206.136, 173.194.206.190, 173.194.206.91, ...
Connecting to dl.google.com (dl.google.com)|173.194.206.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121093268 (115M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.33’

google-chrome-stabl 100%[===================>] 115.48M   244MB/s    in 0.5s    

2026-02-17 16:57:17 (244 MB/s) - ‘google-chrome-stable_current_amd64.deb.33’ saved [121093268/121093268]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
google-chrome-stable is already the newest version (145.0.7632.75-1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


## 2. Automação para fazer uma consulta no site do Diário Oficial da União

Vamos buscar pelo termo "material didático" para levantar os contratos firmados pelo Ministério da Educação na aquisição do item em 2025.

In [48]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager #faz a gestão do driver automaticamente
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurações para rodar no ambiente do Colab
chrome_options = Options()
chrome_options.add_argument('--headless') # Sem interface gráfica
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--remote-debugging-port=9222') # Ajuda na estabilidade no Colab

# O WebDriverManager baixa e configura o driver compatível automaticamente
service = Service(ChromeDriverManager().install())

try:
    driver = webdriver.Chrome(service=service, options=chrome_options)

    print("Abrindo o browser...")
    driver.get("https://in.gov.br/acesso-a-informacao/dados-abertos/base-de-dados")

    print(f"Página carregada: {driver.title}")

except Exception as e:
    print(f"Erro:\n{e}")

Abrindo o browser...
Página carregada: Base de Dados de Publicações do DOU - Imprensa Nacional


In [49]:
#Configurar um tempo de espera para o carregamento dos elementos no DOM
wait = WebDriverWait(driver, 30)

In [50]:
#Buscar documentos por palavra-chave
#input_pesquisa = driver.find_element("id", "search-bar")

input_pesquisa = wait.until(
    EC.presence_of_element_located((By.ID, "search-bar"))
)
input_pesquisa.clear()
input_pesquisa.send_keys("material didático")

In [51]:
#Clicar em Pesquisa Avançada
btn_pesquisa_avancada = wait.until(
    EC.element_to_be_clickable((By.ID, "toggle-search-advanced"))
)
btn_pesquisa_avancada.click()

In [52]:
#Selecionar a opção "Resultado Exato"
input_resultado_exato = wait.until(
    EC.element_to_be_clickable((
        By.ID, "tipo-pesquisa-1"
    ))
)
input_resultado_exato.click()

In [53]:
#Selecionar a opção "personalizado"
input_personalizado = wait.until(
    EC.element_to_be_clickable((
        By.ID, "personalizado"
    ))
)
input_personalizado.click()

#Clicar no input "início"
input_data_inicio = wait.until(
    EC.element_to_be_clickable((
        By.ID, "data-inicio"
    ))
)
input_data_inicio.click()

#Escolher a data 01/01/2025 no calendário
for i in range(0,13): #mudar para 14 no workshop
    btn_anterior = wait.until(
        EC.element_to_be_clickable((
            By.XPATH, "//a[@data-handler='prev']"
        ))
    )
    btn_anterior.click()

data_inicial = wait.until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='ui-state-default' and normalize-space(text())='1']"))
)
data_inicial.click()

#Clicar no input "fim"
input_data_fim = wait.until(
    EC.element_to_be_clickable((
        By.ID, "data-fim"
    ))
)
input_data_fim.click()

#Escolher a data 31/12/2025 no calendário
for i in range(0, 2):
    btn_anterior = wait.until(
        EC.element_to_be_clickable((
            By.XPATH, "//a[@data-handler='prev']"
        ))
    )
    btn_anterior.click()

data_final = wait.until(
    EC.element_to_be_clickable((By.XPATH, "//a[@class='ui-state-default' and normalize-space(text())='31']"))
)
data_final.click()

In [54]:
#Gambiarra necessária para que o date picker funcione corretamente no Colab

# “O site usa um datepicker que só dispara o evento quando há interação humana.
# Em ambiente headless, esse evento às vezes não é disparado.
# Então precisamos sincronizar o valor visual com o valor real do formulário.”

driver.execute_script("""
document.getElementById('data-inicio').value = '01/01/2025';
document.getElementById('data-fim').value = '31/12/2025';
""")

driver.execute_script("""
document.getElementById('data-inicio').dispatchEvent(new Event('change'));
document.getElementById('data-fim').dispatchEvent(new Event('change'));
""")

In [55]:
#Selecioar a opção "seção 3" (compras, contratos, aditivos, convênios, licitações etc)
input_secao3 = wait.until(
    EC.element_to_be_clickable((
        By.ID, "do3"
    ))
)
input_secao3.click()

In [56]:
#Clicar em Pesquisar
pesquisar = wait.until(
    EC.element_to_be_clickable((
        By.XPATH, "//button[normalize-space()='PESQUISAR']"
    ))
)
pesquisar.click()

In [57]:
#Mais filtros... Clicar em "Tipo de Ato"
btn_tipo = wait.until(
    EC.element_to_be_clickable((By.ID, "artTypeAction"))
)
btn_tipo.click()

#Executar o script para selecionar o filtro "Extrato de Contrato"
#Este select é um componente que não expõe as opções no DOM — somente via JS.
driver.execute_script("""
    updateFacet('artType', 'Extrato de Contrato', '');
""")

In [58]:
#Clicar em "Organização Principal"
btn_tipo = wait.until(
    EC.element_to_be_clickable((By.ID, "orgPrinAction"))
)
btn_tipo.click()

#Executar o script para selecionar o filtro "Ministério da Educação"
driver.execute_script("""
    updateFacet('orgPrin', 'Ministério da Educação', '');
""")

In [59]:
#Validar que a busca retornou resultados
resultados = wait.until(
    EC.element_to_be_clickable((
        By.CSS_SELECTOR, "p.search-total-label"
    ))
)

texto = resultados.text.strip()
print(texto)

479 resultados para "material didático"


## 3. Coleta dos dados encontrados

A consulta retornou uma lista paginada. Será necessário criar uma rotina para extrair os dados de cada página

In [60]:
#Armazenar lista com todos os links obtidos dos contratos na página
links = wait.until(
    EC.presence_of_all_elements_located((
        By.XPATH, "//div[@class='resultados-wrapper']//a"
    ))
)

In [61]:
#Importar biblioteca de regex
import re

In [62]:
#Criar objeto de regex para extrairmos os dados que queremos de cada contrato
CAMPOS = {
    "processo": r"Nº Processo:\s*(.*?)(?=Inexigibilidade|Contratante:)",
    "contratante": r"Contratante:\s*(.*?)(?=Contratado:)",
    "contratado": r"Contratado:\s*(.*?)(?=Objeto:)",
    "objeto": r"Objeto:\s*(.*?)(?=Fundamento Legal:|Vigência:)",
    "vigencia": r"Vigência:\s*(.*?)(?=Valor Total:)",
    "valor_total": r"Valor Total:\s*(.*?)(?=Data de Assinatura:)",
    "data_assinatura": r"Data de Assinatura:\s*(.*?)(?=\(|$)"
}

In [63]:
#Lista para armazenarmos cada resultado
registros = []

In [64]:
#Rotina para formatar os dados capturados conforme o padrão do regex informado
def extrair_campos(texto):
    dados = {}
    for campo, padrao in CAMPOS.items():
        registro = re.search(padrao, texto, re.IGNORECASE | re.DOTALL)
        if registro:
            dados[campo] = registro.group(1).strip().rstrip(".")
    return dados

In [65]:
#Clicar em cada link, salvando os dados de cada contrato na lista de registros
def clicar_links_pagina():
  #Obtem os elementos que possuem os links dos contratos
  links_contratos = wait.until(
    EC.presence_of_all_elements_located((
        By.XPATH, "//div[@class='resultados-wrapper']//a"
    ))
  )

  #Itera em cada elemento para clicar no link do contrato
  for i in range(len(links_contratos)):
    links = wait.until(
      EC.presence_of_all_elements_located((
          By.XPATH, "//div[@class='resultados-wrapper']//a"
      ))
    ) #sim, precisamos coletar de novo a cada iteração os links dos contratos
    link = links[i]

    #Abre uma nova página e aguarda 30 milisegundos para carregar textos na tela
    ActionChains(driver)\
        .move_to_element(link)\
        .pause(0.3)\
        .click()\
        .perform()

    #Extraindo dados dos documentos
    paragrafos = driver.find_elements(By.CSS_SELECTOR, "p.dou-paragraph")
    texto = " ".join(p.text for p in paragrafos)
    dados = extrair_campos(texto)
    registros.append(dados)

    #Fecha a página atual e volta à página principal de consulta
    driver.back()

    #Garante que os resultados foram carregados corretamente antes de clicar no próximo link
    wait.until(EC.presence_of_all_elements_located(
        (By.XPATH, "//div[@class='resultados-wrapper']//a")
    ))

clicar_links_pagina()

In [66]:
for registro in registros:
  print(registro)

{'processo': '23034.031501/2023-41', 'contratante': 'FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO', 'contratado': '04.486.030/0001-00 - LUCCA CULTURA LTDA', 'objeto': 'Aquisição de obras literárias no âmbito do programa nacional do livro e do material didático - pnld 2023, objeto 03, nos formatos impresso, html, pdf e vídeo tutorial, destinadas aos estudantes e professores dos anos iniciais do ensino fundamental (1º ao 5º ano), das escolas da educação básica pública, das redes federal, estaduais, municipais e do distrito federal', 'vigencia': '15/12/2025 a 10/12/2026', 'valor_total': 'R$ 290.282,97', 'data_assinatura': '15/12/2025'}
{'processo': '23034.025122/2023-11', 'contratante': 'FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO', 'contratado': '22.622.213/0001-98 - BAMBOLE EDITORA E LIVRARIA LTDA', 'objeto': 'Aquisição de obras literárias no âmbito do programa nacional do livro e do material didático - pnld 2023, objeto 03, nos formatos impresso, html, pdf e vídeo tutorial, destinad

Legal, mas... precisamos fazer essa mesma rotina para as 24 páginas de resultados que obtivemos com a consulta...

In [67]:
#Limpar a lista de resultados
registros = []

In [68]:
#Descobrir qual é a última página de resultado
def ultima_pagina():
  btn_ultima_pagina = wait.until(
      EC.element_to_be_clickable((By.ID, "lastPage" ))
  )
  ultima_pagina = int(btn_ultima_pagina.text)
  #print(f"Última página de resultados: {ultima_pagina}")
  return ultima_pagina

In [69]:
#Rotina para iterar da primeira até a última página de resultados, capturando os textos de cada contrato
def extrair_resultados_todas_pgs():
    total = ultima_pagina()

    for pagina in range(1, total + 1):
        print(f"Indo para a página {pagina}")

        # Coleta os contratos da página atual
        clicar_links_pagina()

        # Se ainda não é a última página, vai para a próxima
        if pagina < total:
            btn_proximo = wait.until(
                EC.element_to_be_clickable((
                    By.ID, "rightArrow"
                ))
            )

            # driver.execute_script(
            #     "arguments[0].scrollIntoView({block:'center'});",
            #     btn_proximo
            # )
            btn_proximo.click()

            # Aguarda os resultados da próxima página carregarem
            wait.until(
                EC.presence_of_element_located((
                    By.XPATH, "//div[@class='resultados-wrapper']//a"
                ))
            )

extrair_resultados_todas_pgs()

Indo para a página 1
Indo para a página 2
Indo para a página 3
Indo para a página 4
Indo para a página 5
Indo para a página 6
Indo para a página 7
Indo para a página 8
Indo para a página 9
Indo para a página 10
Indo para a página 11
Indo para a página 12
Indo para a página 13
Indo para a página 14
Indo para a página 15
Indo para a página 16
Indo para a página 17
Indo para a página 18
Indo para a página 19
Indo para a página 20
Indo para a página 21
Indo para a página 22
Indo para a página 23
Indo para a página 24


In [72]:
print(f"Total de contratos: {len(registros)}")

Total de contratos: 479


In [73]:
driver.quit()

# 4. Salvar resultados em arquivo .csv

In [74]:
#Importar a biblioteca Pandas
import pandas as pd

In [75]:
#Cria um DataFrame e salva o resultado como arquivo .csv
df = pd.DataFrame(registros)
df.to_csv('gastos_material_didatico_mec_2025.csv', index=False, encoding='utf-8')

In [76]:
df.head()

,processo,contratante,contratado,objeto,vigencia,valor_total,data_assinatura
0,23034.031501/2023-41,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,04.486.030/0001-00 - LUCCA CULTURA LTDA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,"R$ 290.282,97",15/12/2025
1,23034.025122/2023-11,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,22.622.213/0001-98 - BAMBOLE EDITORA E LIVRARI...,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,"R$ 725.941,01",15/12/2025
2,23034.031153/2023-10,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,20.513.288/0001-05 - LAG NEIRA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,"R$ 585.684,78",15/12/2025
3,23034.031498/2023-65,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,43.401.056/0001-60 - LONGUINA EDITORA LTDA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,"R$ 242.918,22",15/12/2025
4,23034.031539/2023-13,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,08.675.034/0001-98 - SCOPPIO EDITORA LTDA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,"R$ 201.005,82",15/12/2025


## 5. Agora você pode analisar os dados da maneira que quiser =)

In [79]:
import math

def limpar_valor(valor):
    if valor is None:
        return None

    # Se já for int, retorna float do valor
    if isinstance(valor, (int, float)):
        if math.isnan(valor):
            return None
        return float(valor)

    # Se for string, transforma em float
    valor = str(valor)

    return float(
        valor.replace("R$", "")
             .replace(".", "")
             .replace(",", ".")
             .strip()
    )

In [80]:
df["valor_total"] = df["valor_total"].apply(limpar_valor)

In [81]:
df.head()

,processo,contratante,contratado,objeto,vigencia,valor_total,data_assinatura
0,23034.031501/2023-41,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,04.486.030/0001-00 - LUCCA CULTURA LTDA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,290282.97,15/12/2025
1,23034.025122/2023-11,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,22.622.213/0001-98 - BAMBOLE EDITORA E LIVRARI...,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,725941.01,15/12/2025
2,23034.031153/2023-10,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,20.513.288/0001-05 - LAG NEIRA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,585684.78,15/12/2025
3,23034.031498/2023-65,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,43.401.056/0001-60 - LONGUINA EDITORA LTDA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,242918.22,15/12/2025
4,23034.031539/2023-13,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCACAO,08.675.034/0001-98 - SCOPPIO EDITORA LTDA,Aquisição de obras literárias no âmbito do pro...,15/12/2025 a 10/12/2026,201005.82,15/12/2025


In [83]:
valor_contratos_2025 = df['valor_total'].sum()
print(f"Em 2025, o MEC adquiriu R$ {valor_contratos_2025} em materiais didáticos")

Em 2025, o MEC adquiriu R$ 1044311365.78 em materiais didáticos


In [89]:
top_10 = df.groupby("contratado")["valor_total"].sum().sort_values(ascending=False).head(10)

In [90]:
top_10_formatado = top_10.apply(
    lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
)

top_10_formatado.head(10)

,valor_total
contratado,
61.186.490/0001-57 - EDITORA FTD S A,"R$ 224.304.108,38"
62.136.304/0001-38 - EDITORA MODERNA LTDA,"R$ 166.349.489,09"
61.259.958/0001-96 - EDITORA ATICA S.A,"R$ 141.299.720,82"
60.657.574/0001-69 - EDITORA DO BRASIL SA,"R$ 100.961.692,23"
05.699.378/0001-49 - EDICOES SM LTDA,"R$ 60.189.785,58"
44.127.355/0001-11 - EDITORA SCIPIONE S.A,"R$ 41.837.200,27"
50.268.838/0001-39 - SARAIVA EDUCACAO S.A,"R$ 36.419.411,85"
61.097.432/0001-57 - KIT'S EDITORA COMERCIO E INDUSTRIA LTDA,"R$ 12.534.325,15"
15.194.990/0001-13 - UNIVERSO EDUCACAO LTDA,"R$ 11.838.856,64"
